#Proyecto Final - Pruebas
- *Luis Esteban Centeno Untivero*

## Instalando liberías a utilizar:

In [35]:
%pip install -qU langgraph langchain-openai langchain-core langchain-community langchain-text-splitters langchain-elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.3/906.3 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.9 MB/s eta 0:00:00


# Creación del RAG:

In [2]:
from google.colab import userdata
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
import json


os.environ["OPENAI_API_KEY"] = userdata.get('OPENIA_TOKEN')

In [4]:
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [7]:
# Menú de prueba a utilizar:

# menu = [
#   {
#     "nombre": "Hamburguesa Clásica",
#     "descripcion": "Pan artesanal, carne de res, lechuga, tomate, cebolla y aderezo especial.",
#     "precio": 8.5,
#     "ingredientes": ["carne", "pan", "lechuga", "tomate", "cebolla", "aderezo"],
#     "tags": ["carne", "combo", "almuerzo"],
#     "disponibilidad": True,
#     "categoria": "plato principal"
#   },
#   {
#     "nombre": "Ensalada Vegana",
#     "descripcion": "Mezcla de hojas verdes, quinoa, garbanzos, tomate cherry, aguacate y vinagreta de limón.",
#     "precio": 7.0,
#     "ingredientes": ["quinoa", "garbanzos", "aguacate", "lechuga", "vinagreta"],
#     "tags": ["vegano", "sin gluten", "saludable"],
#     "disponibilidad": True,
#     "categoria": "plato principal"
#   },
#   {
#     "nombre": "Tacos de Pollo",
#     "descripcion": "Tres tacos con pollo sazonado, cebolla morada, cilantro y salsa verde.",
#     "precio": 6.5,
#     "ingredientes": ["pollo", "tortilla", "cebolla", "cilantro", "salsa"],
#     "tags": ["pollo", "picante"],
#     "disponibilidad": False,
#     "categoria": "plato principal"
#   },
#   {
#     "nombre": "Pizza Margarita",
#     "descripcion": "Pizza con base de tomate, mozzarella fresca y albahaca.",
#     "precio": 9.0,
#     "ingredientes": ["queso", "tomate", "masa", "albahaca"],
#     "tags": ["vegetariano"],
#     "disponibilidad": True,
#     "categoria": "plato principal"
#   },
#   {
#     "nombre": "Brownie con Helado",
#     "descripcion": "Brownie de chocolate tibio con bola de helado de vainilla.",
#     "precio": 4.0,
#     "ingredientes": ["chocolate", "harina", "azúcar", "helado"],
#     "tags": ["postre", "dulce"],
#     "disponibilidad": True,
#     "categoria": "postre"
#   },
#   {
#     "nombre": "Limonada Natural",
#     "descripcion": "Bebida refrescante con jugo de limón, menta y hielo.",
#     "precio": 2.5,
#     "ingredientes": ["limón", "agua", "menta", "azúcar"],
#     "tags": ["bebida", "natural"],
#     "disponibilidad": True,
#     "categoria": "bebida"
#   },
#   {
#     "nombre": "Combo Familiar",
#     "descripcion": "2 hamburguesas, 2 refrescos, papas grandes y 1 postre.",
#     "precio": 18.0,
#     "ingredientes": ["carne", "pan", "papas", "refresco", "postre"],
#     "tags": ["combo", "familiar", "ahorro"],
#     "disponibilidad": True,
#     "categoria": "combo",
#     "cupon": "FAMILIA20"
#   }
# ]


In [9]:
# Lo guardamos como archivo

with open("menu.json", "w", encoding="utf-8") as f:
    json.dump(menu, f, ensure_ascii=False, indent=4)

In [19]:
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Cargar el archivo JSON
with open("menu.json", "r", encoding="utf-8") as f:
    menu_data = json.load(f)

# Convertir cada plato en documento con metadata
docs = []

for item in menu_data:
    contenido = f"{item['nombre']}: {item['descripcion']} Precio: ${item['precio']}"
    doc = Document(
        page_content=contenido,
        metadata={
            "nombre": item["nombre"],
            "precio": item["precio"],
            "categoria": item["categoria"],
            "tags": item.get("tags", []),
            "disponibilidad": item.get("disponibilidad", True),
            "cupon": item.get("cupon", None)
        }
    )
    docs.append(doc)

In [20]:
docs

[Document(metadata={'nombre': 'Hamburguesa Clásica', 'precio': 8.5, 'categoria': 'plato principal', 'tags': ['carne', 'combo', 'almuerzo'], 'disponibilidad': True, 'cupon': None}, page_content='Hamburguesa Clásica: Pan artesanal, carne de res, lechuga, tomate, cebolla y aderezo especial. Precio: $8.5'),
 Document(metadata={'nombre': 'Ensalada Vegana', 'precio': 7.0, 'categoria': 'plato principal', 'tags': ['vegano', 'sin gluten', 'saludable'], 'disponibilidad': True, 'cupon': None}, page_content='Ensalada Vegana: Mezcla de hojas verdes, quinoa, garbanzos, tomate cherry, aguacate y vinagreta de limón. Precio: $7.0'),
 Document(metadata={'nombre': 'Tacos de Pollo', 'precio': 6.5, 'categoria': 'plato principal', 'tags': ['pollo', 'picante'], 'disponibilidad': False, 'cupon': None}, page_content='Tacos de Pollo: Tres tacos con pollo sazonado, cebolla morada, cilantro y salsa verde. Precio: $6.5'),
 Document(metadata={'nombre': 'Pizza Margarita', 'precio': 9.0, 'categoria': 'plato principal

In [23]:
# Primero hacemos una prueba con memoria local:

vectore_store = InMemoryVectorStore(embedding)
_ = vectore_store.add_documents(docs)

In [24]:
retriever_chain = vectore_store.as_retriever()

In [25]:
# Pregunta de prueba:

query= "Cual es el precio de la hamburgeua clásica?"
respuesta = retriever_chain.invoke(query)
respuesta

[Document(id='20894b9b-152d-4b88-a94a-ff54e0a98b47', metadata={'nombre': 'Hamburguesa Clásica', 'precio': 8.5, 'categoria': 'plato principal', 'tags': ['carne', 'combo', 'almuerzo'], 'disponibilidad': True, 'cupon': None}, page_content='Hamburguesa Clásica: Pan artesanal, carne de res, lechuga, tomate, cebolla y aderezo especial. Precio: $8.5'),
 Document(id='844c6c61-2fa8-4f81-99f7-51d299e597a5', metadata={'nombre': 'Combo Familiar', 'precio': 18.0, 'categoria': 'combo', 'tags': ['combo', 'familiar', 'ahorro'], 'disponibilidad': True, 'cupon': 'FAMILIA20'}, page_content='Combo Familiar: 2 hamburguesas, 2 refrescos, papas grandes y 1 postre. Precio: $18.0'),
 Document(id='990e2b26-71a8-4731-a772-422ad65dbbb2', metadata={'nombre': 'Pizza Margarita', 'precio': 9.0, 'categoria': 'plato principal', 'tags': ['vegetariano'], 'disponibilidad': True, 'cupon': None}, page_content='Pizza Margarita: Pizza con base de tomate, mozzarella fresca y albahaca. Precio: $9.0'),
 Document(id='74a2a55d-74f

In [26]:
# Convertimos el RAG como tool:

tool_menu = retriever_chain.as_tool(
    name ="datos_menu",
    description="Herramienta que tiene la información detallada (nombre, descripcion, precio, ingredientes, disponibilidad y categoria) de todos los productosdel menú"
)

/tmp/ipython-input-26-2679142047.py:1: LangChainBetaWarning: This API is in beta and may change in the future.
  tool_menu = retriever_chain.as_tool(


In [30]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# Pruena de concepto:

toolkit=[tool_menu]

model = ChatOpenAI(model="gpt-4.1-2025-04-14")
agent = create_react_agent(model, toolkit)

# Invocamos al agente
config = {"configurable": {"thread_id": "000"}}
response = agent.invoke({"messages": [HumanMessage(content="Que comidas saludables hay en el menú?")]}, config=config)
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

En el menú hay estas opciones saludables:

1. Ensalada Vegana: Mezcla de hojas verdes, quinoa, garbanzos, tomate cherry, aguacate y vinagreta de limón. Precio: $7.0

También puedes considerar la Limonada Natural como una bebida fresca y natural: Jugo de limón, menta y hielo. Precio: $2.5

¿Te gustaría saber más detalles sobre alguna de estas opciones o ver alternativas?


## Pasamos el RAG a ElactiSearch

In [36]:
from langchain.vectorstores import ElasticsearchStore

In [39]:
# Definimos nuestro entorno:
db = ElasticsearchStore.from_documents(
    docs,
    embedding,
    es_url="",
    es_user="elastic",
    es_password="",
    index_name="menu-data",
)

Leemos los datos del Elastic para comprobar la funcionalidad:

In [55]:
db_query= ElasticsearchStore(
    es_url="",
    es_user="elastic",
    es_password= "",
    index_name= "menu-data",
    embedding=embedding)

# Transformamos nuestro RAG como tool:
retriever = db_query.as_retriever()

tool_rag =retriever.as_tool(
    name ="datos_menu",
    description="Herramienta que tiene la información detallada (nombre, descripcion, precio, ingredientes, disponibilidad y categoria) de todos los productosdel menú"
)

In [44]:
  #inicializamos el agente
  toolkit = [tool_rag]
  agent_executor = create_react_agent(model, toolkit)

  config = {"configurable": {"thread_id": "mc0001"}} ## el thread_id es el identificador de memoria
  for step in agent_executor.stream(
      {"messages": [HumanMessage(content="Que platos están disponibles ahora?")]},
      config,
      stream_mode="values",
  ):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Que platos están disponibles ahora?
================================== Ai Message ==================================
Tool Calls:
  datos_menu (call_eLa0q1jI6MX1nX6e0t0q5L11)
 Call ID: call_eLa0q1jI6MX1nX6e0t0q5L11
  Args:
    __arg1: disponible
================================= Tool Message =================================
Name: datos_menu

[Document(metadata={'nombre': 'Ensalada Vegana', 'precio': 7.0, 'categoria': 'plato principal', 'tags': ['vegano', 'sin gluten', 'saludable'], 'disponibilidad': True, 'cupon': None}, page_content='Ensalada Vegana: Mezcla de hojas verdes, quinoa, garbanzos, tomate cherry, aguacate y vinagreta de limón. Precio: $7.0'), Document(metadata={'nombre': 'Limonada Natural', 'precio': 2.5, 'categoria': 'bebida', 'tags': ['bebida', 'natural'], 'disponibilidad': True, 'cupon': None}, page_content='Limonada Natural: Bebida refrescante con jugo de limón, menta y hielo. Precio: $2.5

# Creación  y prueba de los tools:

Tool - Stock:

In [45]:
stock_actual = {
    "Hamburguesa Clásica": 4,
    "Tacos al Pastor": 0,
    "Ensalada Vegana": 10,
    "Pizza Margarita": 2,
    "Smoothie de Frutas": 0,
}

In [46]:
from langchain.agents import tool

@tool
def verificar_stock(nombre_plato: str) -> str:
    """
    Verifica la si un plato está disponible en el menú y la cantidad disponible del plato.
    """
    nombre_plato = nombre_plato.strip()
    if nombre_plato in stock_actual:
        cantidad = stock_actual[nombre_plato]
        if cantidad > 0:
            return f"✅ El plato '{nombre_plato}' está disponible. Quedan {cantidad} unidades."
        else:
            return f"❌ Lo siento, el plato '{nombre_plato}' no está disponible actualmente."
    else:
        return f"🤔 No encontré el plato '{nombre_plato}' en el inventario."


In [52]:
  #inicializamos el agente
  toolkit = [tool_rag, verificar_stock]
  agent_executor = create_react_agent(model, toolkit)

  config = {"configurable": {"thread_id": "mc0001"}} ## el thread_id es el identificador de memoria
  for step in agent_executor.stream(
      {"messages": [HumanMessage(content="Cuantos Tacos al Pastor te quedan?")]},
      config,
      stream_mode="values",
  ):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Cuantos Tacos al Pastor te quedan?
================================== Ai Message ==================================
Tool Calls:
  verificar_stock (call_vQ0N0y81UHo6AzBRsz9TlXec)
 Call ID: call_vQ0N0y81UHo6AzBRsz9TlXec
  Args:
    nombre_plato: Tacos al Pastor
================================= Tool Message =================================
Name: verificar_stock

❌ Lo siento, el plato 'Tacos al Pastor' no está disponible actualmente.
================================== Ai Message ==================================

Actualmente, los Tacos al Pastor no están disponibles en el menú. ¿Te gustaría conocer otras opciones similares?


Tool - Recomendación

In [58]:
@tool
def recomendar_similar(plato_input: str) -> str:
    """
    Recomienda platos similares al que el usuario menciona.
    """
    retriever = db_query.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    resultados = retriever.get_relevant_documents(plato_input)

    if not resultados:
        return "No encontré platos similares 😕"

    recomendaciones = [doc.metadata['nombre'] for doc in resultados]
    return f"Basado en lo que mencionaste, podrías probar: {', '.join(recomendaciones)}"


In [59]:
  #inicializamos el agente
  toolkit = [tool_rag, verificar_stock, recomendar_similar]
  agent_executor = create_react_agent(model, toolkit)

  config = {"configurable": {"thread_id": "mc0001"}} ## el thread_id es el identificador de memoria
  for step in agent_executor.stream(
      {"messages": [HumanMessage(content="No quiero el combo familiar porque es mucho para mi, que otra opción del menú me recomiendas?")]},
      config,
      stream_mode="values",
  ):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

No quiero el combo familiar porque es mucho para mi, que otra opción del menú me recomiendas?
================================== Ai Message ==================================
Tool Calls:
  recomendar_similar (call_IrArizSUAOhIcU3Lzq2Tc2yj)
 Call ID: call_IrArizSUAOhIcU3Lzq2Tc2yj
  Args:
    plato_input: combo familiar


/tmp/ipython-input-58-253156759.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resultados = retriever.get_relevant_documents(plato_input)


================================= Tool Message =================================
Name: recomendar_similar

Basado en lo que mencionaste, podrías probar: Combo Familiar, Ensalada Vegana, Hamburguesa Clásica
================================== Ai Message ==================================
Tool Calls:
  datos_menu (call_GZveuBbC4QtTSZZqrMFyEDfD)
 Call ID: call_GZveuBbC4QtTSZZqrMFyEDfD
  Args:
    __arg1: Ensalada Vegana
  datos_menu (call_we31Zx4TnxsFaJyAzDw4ZnkY)
 Call ID: call_we31Zx4TnxsFaJyAzDw4ZnkY
  Args:
    __arg1: Hamburguesa Clásica
================================= Tool Message =================================
Name: datos_menu

[Document(metadata={'nombre': 'Hamburguesa Clásica', 'precio': 8.5, 'categoria': 'plato principal', 'tags': ['carne', 'combo', 'almuerzo'], 'disponibilidad': True, 'cupon': None}, page_content='Hamburguesa Clásica: Pan artesanal, carne de res, lechuga, tomate, cebolla y aderezo especial. Precio: $8.5'), Document(metadata={'nombre': 'Combo Familiar', 'pre

In [61]:
  #inicializamos el agente
  toolkit = [tool_rag, verificar_stock, recomendar_similar]
  agent_executor = create_react_agent(model, toolkit)

  config = {"configurable": {"thread_id": "mc0001"}} ## el thread_id es el identificador de memoria
  for step in agent_executor.stream(
      {"messages": [HumanMessage(content="Solo tengo 7$ que puedo comprar?")]},
      config,
      stream_mode="values",
  ):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Solo tengo 7$ que puedo comprar?
================================== Ai Message ==================================
Tool Calls:
  datos_menu (call_g49mPGye4p4tcSsSOsRFv45S)
 Call ID: call_g49mPGye4p4tcSsSOsRFv45S
  Args:
    __arg1: todo
================================= Tool Message =================================
Name: datos_menu

[Document(metadata={'nombre': 'Pizza Margarita', 'precio': 9.0, 'categoria': 'plato principal', 'tags': ['vegetariano'], 'disponibilidad': True, 'cupon': None}, page_content='Pizza Margarita: Pizza con base de tomate, mozzarella fresca y albahaca. Precio: $9.0'), Document(metadata={'nombre': 'Combo Familiar', 'precio': 18.0, 'categoria': 'combo', 'tags': ['combo', 'familiar', 'ahorro'], 'disponibilidad': True, 'cupon': 'FAMILIA20'}, page_content='Combo Familiar: 2 hamburguesas, 2 refrescos, papas grandes y 1 postre. Precio: $18.0'), Document(metadata={'nombre': 'Tacos de Pollo

Tool - Cupones:

In [62]:
cupones_validos = {
    "FAMILIA20": 0.20,
    "BEBIDA10": 0.10,
    "VEGANO15": 0.15,
}

@tool
def aplicar_cupon(nombre_cupon: str, total: float) -> str:
    """
    Aplica un cupón de descuento al total si es válido.
    """
    nombre_cupon = nombre_cupon.strip().upper()
    if nombre_cupon in cupones_validos:
        descuento = cupones_validos[nombre_cupon]
        nuevo_total = round(total * (1 - descuento), 2)
        return f"✅ Cupón aplicado. Descuento del {int(descuento*100)}%. Total final: ${nuevo_total}"
    else:
        return f"❌ El cupón '{nombre_cupon}' no es válido."


In [65]:
  #inicializamos el agente
  toolkit = [tool_rag, verificar_stock, recomendar_similar, aplicar_cupon]
  agent_executor = create_react_agent(model, toolkit)

  config = {"configurable": {"thread_id": "mc0001"}} ## el thread_id es el identificador de memoria
  for step in agent_executor.stream(
      {"messages": [HumanMessage(content="Quiero comprar un Combo Familiar y una Hamburgueas Cláica y tengo un cupon: BEBIDA10, cuanto me saldría todo?")]},
      config,
      stream_mode="values",
  ):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Quiero comprar un Combo Familiar y una Hamburgueas Cláica y tengo un cupon: BEBIDA10, cuanto me saldría todo?
================================== Ai Message ==================================
Tool Calls:
  datos_menu (call_mjbFuWCT45if7In2OYA4YI96)
 Call ID: call_mjbFuWCT45if7In2OYA4YI96
  Args:
    __arg1: Combo Familiar
  datos_menu (call_scsi8mvo73Q0TSFMZJmUdxm1)
 Call ID: call_scsi8mvo73Q0TSFMZJmUdxm1
  Args:
    __arg1: Hamburguesa Clásica
================================= Tool Message =================================
Name: datos_menu

[Document(metadata={'nombre': 'Hamburguesa Clásica', 'precio': 8.5, 'categoria': 'plato principal', 'tags': ['carne', 'combo', 'almuerzo'], 'disponibilidad': True, 'cupon': None}, page_content='Hamburguesa Clásica: Pan artesanal, carne de res, lechuga, tomate, cebolla y aderezo especial. Precio: $8.5'), Document(metadata={'nombre': 'Combo Familiar', 'precio': 18.0, 'ca

# Configuración de memoria:

In [68]:
%pip install -qU langchain langchain_openai langchain-core langgraph langchain-community psycopg[binary,pool]==3.2.6 langgraph-checkpoint-postgres langchain-elasticsearch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.8 MB/s eta 0:00:00


In [85]:
from langchain_core.prompts import ChatPromptTemplate

#Variables de memoria
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
# Inicializamos la memoria
with ConnectionPool(
    # Example configuration
    conninfo='',
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    # Inicializamos el modelo sde recomienda el 4.1
    model = ChatOpenAI(model="gpt-4.1-2025-04-14")

    # Agrupamos las herramientas
    tolkit = [tool_rag]

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
            """
            Eres un asistente virtual amigable y accesible especializado en guiar a personas con discapacidad visual a través del menú de un restaurante.
            Utiliza únicamente las herramientas disponibles para responder: consulta de menú (RAG), verificación de stock, aplicación de cupones y recomendación de platos similares. Si no cuentas con una herramienta específica para responder una pregunta, indícalo de forma clara y sugiere cómo podrías ayudar.
            Tu objetivo es que el cliente se sienta acompañado, informado y en control de su pedido. Sé breve, claro, cálido y evita tecnicismos innecesarios. Eres paciente, empático y actúas como un asistente personal que lo guía en todo momento.
            Sigue esta guía de conversación:
            1. **Saludo y apertura:**
            Da una bienvenida cordial. Pregunta si el cliente ya tiene en mente algún plato o si quiere que le leas opciones del menú por categoría (ej. entradas, principales, postres, vegetarianos, etc.).
            2. **Consulta del menú:*
            Usa la herramienta de menú para brindar información precisa. Describe brevemente cada plato mencionado: nombre, ingredientes principales, precio, y si forma parte de algún combo. Si el cliente tiene preferencias (ej. sin carne, sin lactosa), sugiere opciones que se alineen.
            3. **Verificación de disponibilidad:**
            Antes de confirmar un pedido, verifica si los platos solicitados están disponibles en stock. Si algo no está disponible, sugiere una alternativa similar usando la herramienta de recomendación.
            4. **Aplicación de cupones o promociones:**
            Si el cliente menciona un cupón o desea saber si hay promociones, usa la herramienta para aplicar descuentos o combos y muestra el precio actualizado.
            5. **Resumen del pedido:**
            Repite lo que ha seleccionado el cliente con los precios individuales y el total final. Pregunta si desea añadir algo más antes de cerrar el pedido.
            6. **Cierre de atención:**
            Agradece su tiempo y disponibilidad. Indícale cómo puede proceder para finalizar el pedido (ej. llamar al restaurante, pagar presencialmente o pedir ayuda en el local). Si el cliente necesita ayuda adicional, ofrécesela de inmediato.
            7. **Estilo de comunicación:**
            Sé cercano, conversacional y directo. Mantén frases cortas y claras. Si el cliente no entiende algo o parece confundido, repite o simplifica tu respuesta. Tu prioridad es la comodidad y accesibilidad del cliente.

            """),
        ("human", "{messages}"),
        ]
    )
    #inicializamos el agente
    agent_executor = create_react_agent(model, tolkit, checkpointer=checkpointer, prompt=prompt)


    config = {"configurable": {"thread_id": "mc0001"}} ## el thread_id es el identificador de memoria
    for step in agent_executor.stream(
        {"messages": [HumanMessage(content="Aplica el cupón que dices")]},
        config,
        stream_mode="values",
    ):
      step["messages"][-1].pretty_print()

================================ Human Message =================================

Aplica el cupón que dices
================================== Ai Message ==================================

¡Excelente elección! 👏 El cupón FAMILIA20 aplica para el Combo Familiar y te da un descuento especial.

Aquí te resumo tu pedido actualizado:

- Combo Familiar: S/18 (con cupón FAMILIA20, ¡tendrá descuento!)
- Hamburguesa Clásica: S/8.5

Aplicando el cupón FAMILIA20, el Combo Familiar tendría un precio menor (te avisaré el monto exacto apenas se procese). El total será un poco menos de S/26.5 gracias al descuento.

¿Te gustaría recoger en tienda o prefieres envío a domicilio? Recuerda, si tu compra supera los S/50 el envío es gratis, si no, el costo es S/20. ¿Te gustaría agregar algo más para acercarte al envío gratis o quieres finalizar así tu pedido?


# Prueba pre Docker:

In [110]:
from langchain_openai import ChatOpenAI
import os
from flask import Flask, jsonify, request
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.prebuilt import create_react_agent


## datos de trazabilidad
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "gcpaiagent"
os.environ["OPENAI_API_KEY"] ="sk"


id_agente = 'idagente'
#datos de configuracion
DB_URI = os.environ.get(
    "DB_URI",
    "postgres://"
)
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
db_query = ElasticsearchStore(
    es_url="",
    es_user="elastic",
    es_password="",
    index_name="menu-data",
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"))

# Herramienta RAG
retriever = db_query.as_retriever()
tool_rag =retriever.as_tool(
name ="datos_menu",
description="Herramienta que tiene la información detallada (nombre, descripcion, precio, ingredientes, disponibilidad y categoria) de todos los productosdel menú"
)

stock_actual = {
"Hamburguesa Clásica": 4,
"Tacos al Pastor": 0,
"Ensalada Vegana": 10,
"Pizza Margarita": 2,
"Smoothie de Frutas": 0,
}

cupones_validos = {
"FAMILIA20": 0.20,
"BEBIDA10": 0.10,
"VEGANO15": 0.15,
}

from langchain.agents import tool

@tool
def verificar_stock(nombre_plato: str) -> str:
    """
    Verifica la si un plato está disponible en el menú y la cantidad disponible del plato.
    """
    nombre_plato = nombre_plato.strip()
    if nombre_plato in stock_actual:
        cantidad = stock_actual[nombre_plato]
        if cantidad > 0:
            return f"✅ El plato '{nombre_plato}' está disponible. Quedan {cantidad} unidades."
        else:
            return f"❌ Lo siento, el plato '{nombre_plato}' no está disponible actualmente."
    else:
        return f"🤔 No encontré el plato '{nombre_plato}' en el inventario."

@tool
def aplicar_cupon(nombre_cupon: str, total: float) -> str:
    """
    Aplica un cupón de descuento al total si es válido.
    """
    nombre_cupon = nombre_cupon.strip().upper()
    if nombre_cupon in cupones_validos:
        descuento = cupones_validos[nombre_cupon]
        nuevo_total = round(total * (1 - descuento), 2)
        return f"✅ Cupón aplicado. Descuento del {int(descuento*100)}%. Total final: ${nuevo_total}"
    else:
        return f"❌ El cupón '{nombre_cupon}' no es válido."


@tool
def notificar_camarero(motivo: str = "Asistencia general") -> str:
    """
    Tool para notificar al camarero que el cliente necesita asistencia.
    Puede indicar el motivo: dificultad visual, confusión, asistencia para ordenar, etc.
    """
    import datetime
    import requests

    hora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    payload = {
        "motivo": motivo,
        "hora": hora,
        "estado": "pendiente"
    }

    # Reemplaza con tu URL de webhook.site
    webhook_url = "	https://webhook.site/64914f9d-304a-4ac9-bb0a-6ce251d91f61"

    try:
        requests.post(webhook_url, json=payload)
        return f"✅ Notificación enviada al camarero. Motivo: '{motivo}' a las {hora}."
    except:
        return "⚠️ No se pudo enviar la notificación. Inténtalo de nuevo."


# Inicializamos la memoria
with ConnectionPool(
        # Example configuration
        conninfo=DB_URI,
        max_size=20,
        kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    # Inicializamos el modelo
    model = ChatOpenAI(model="gpt-4.1-2025-04-14")

    # Agrupamos las herramientas
    tolkit = [tool_rag, verificar_stock, aplicar_cupon, notificar_camarero]

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
              """
            Eres un asistente virtual amigable y accesible especializado en guiar a personas con discapacidad visual a través del menú de un restaurante.
            Utiliza únicamente las herramientas disponibles para responder: consulta de menú (RAG), verificación de stock, aplicación de cupones y recomendación de platos similares. Si no cuentas con una herramienta específica para responder una pregunta, indícalo de forma clara y sugiere cómo podrías ayudar.
            Tu objetivo es que el cliente se sienta acompañado, informado y en control de su pedido. Sé breve, claro, cálido y evita tecnicismos innecesarios. Eres paciente, empático y actúas como un asistente personal que lo guía en todo momento.
            Sigue esta guía de conversación:
            1. **Saludo y apertura:**
            Da una bienvenida cordial. Pregunta si el cliente ya tiene en mente algún plato o si quiere que le leas opciones del menú por categoría (ej. entradas, principales, postres, vegetarianos, etc.).
            2. **Consulta del menú:*
            Usa la herramienta de menú para brindar información precisa. Describe brevemente cada plato mencionado: nombre, ingredientes principales, precio, y si forma parte de algún combo. Si el cliente tiene preferencias (ej. sin carne, sin lactosa), sugiere opciones que se alineen.
            3. **Verificación de disponibilidad:**
            Antes de confirmar un pedido, verifica si los platos solicitados están disponibles en stock. Si algo no está disponible, sugiere una alternativa similar usando la herramienta de recomendación.
            4. **Aplicación de cupones o promociones:**
            Si el cliente menciona un cupón o desea saber si hay promociones, usa la herramienta para aplicar descuentos o combos y muestra el precio actualizado.
            5. **Resumen del pedido:**
            Repite lo que ha seleccionado el cliente con los precios individuales y el total final. Pregunta si desea añadir algo más antes de cerrar el pedido.
            6. **Cierre de atención:**
            Agradece su tiempo y disponibilidad. Indícale cómo puede proceder para finalizar el pedido (ej. llamar al restaurante, pagar presencialmente o pedir ayuda en el local). Si el cliente necesita ayuda adicional, ofrécesela de inmediato.
            7. **Estilo de comunicación:**
            Sé cercano, conversacional y directo. Mantén frases cortas y claras. Si el cliente no entiende algo o parece confundido, repite o simplifica tu respuesta. Tu prioridad es la comodidad y accesibilidad del cliente.

              """),
            ("human", "{messages}"),
        ]
    )
    # inicializamos el agente
    agent_executor = create_react_agent(model, tolkit, checkpointer=checkpointer, prompt=prompt)
    # ejecutamos el agente
    config = {"configurable": {"thread_id": id_agente}}
    response = agent_executor.invoke({"messages": [HumanMessage(content="Hola no entiendo el menú puede venir alguien a ayudarme?")]}, config=config)
    response['messages'][-1].pretty_print()


================================== Ai Message ==================================

¡Gracias por decírmelo! Ya he notificado a un camarero para que venga a ayudarte personalmente. Pronto llegará a tu mesa para asistirte y explicarte el menú en persona.

Si necesitas que te siga acompañando o te ayude con otra consulta mientras esperas, sólo dímelo. Estoy aquí para apoyarte en lo que necesites.


CONVERSIÓN DE VOZ A TEXTO:

In [107]:
client = OpenAI(api_key="")

# Abrimos el audio generado
audio_file = open("Consulta.mp3", "rb")

# Hacemos una transcripción del audio:
msg = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

# msg => Que platos hay disponibles para hoy?

# inicializamos el agente
agent_executor = create_react_agent(model, tolkit, checkpointer=checkpointer, prompt=prompt)

# ejecutamos el agente
config = {"configurable": {"thread_id": id_agente}}
response = agent_executor.invoke({"messages": [HumanMessage(content=msg)]}, config=config)
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

Estos son algunos de los platos disponibles hoy:

- Hamburguesa Clásica: Pan artesanal, carne de res, lechuga, tomate, cebolla y aderezo especial. Precio: 8,50 dólares.
- Combo Familiar: Incluye 2 hamburguesas, 2 refrescos, papas grandes y 1 postre. Precio: 18 dólares. Este combo tiene cupón de descuento llamado FAMILIA20.
- Pizza Margarita: Base de tomate, mozzarella fresca y albahaca. Plato vegetariano. Precio: 9 dólares.
- Ensalada Vegana: Hojas verdes, quinoa, garbanzos, tomate cherry, aguacate y vinagreta de limón. Es vegana y sin gluten. Precio: 7 dólares.

¿Te gustaría saber más detalles sobre alguno de estos platos, o prefieres que busque otras opciones (como entradas, postres o bebidas)? ¡Cuéntame cómo prefieres seguir!


# Texto a voz

In [ ]:
import pyttsx3

# Inicializar el motor de texto a voz
engine = pyttsx3.init()

# Configuración opcional (puedes cambiar el volumen y la velocidad)
engine.setProperty('rate', 150)  # Velocidad del habla (normalmente entre 100-200)
engine.setProperty('volume', 1)  # Volumen (0.0 a 1.0)

# Texto a convertir a voz
texto = response['messages'][-1].pretty_print()

# Hacer que el motor lea el texto
engine.say(texto)

# Esperar a que termine de hablar
engine.runAndWait()